# Imports

In [ ]:
import import_ipynb
import AlexFunctions as AF

import copy
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.measure import label, regionprops

# Visualize Results

In [ ]:
# BOUNDING BOX

def create_bounding_boxes(mission_numbers, anomaly_masks, output_images, num_bbox_iterations=1, bbox_colors=[(0, 0, 255), (255, 0, 0)], voting_scheme='AND', majority_success=1, display_images=False):
    combined_masks = []
    for i in range(0, len(anomaly_masks[0])): # number of differences in mission used = len(mission_numbers)-1
        if voting_scheme == 'AND':
            combined_mask = np.ones((anomaly_masks[0][0].shape[0:2])).astype(bool)
        elif voting_scheme == 'OR':
            combined_mask = np.zeros((anomaly_masks[0][0].shape[0:2])).astype(bool)
        elif voting_scheme == 'MAJORITY':
            combined_mask = np.zeros((anomaly_masks[0][0].shape[0:2])).astype(bool)
        else:
            print('Invalid Voting Scheme Selected')

        for j in range(0, len(anomaly_masks)): # number of detection methods
            if voting_scheme == 'AND':
                combined_mask = np.logical_and(combined_mask, anomaly_masks[j][i].astype(bool))
            elif voting_scheme == 'OR':
                combined_mask = np.logical_or(combined_mask, anomaly_masks[j][i].astype(bool))
            elif voting_scheme == 'MAJORITY':
                combined_mask[anomaly_masks[j][i].astype(bool)] += 1

        if voting_scheme == 'MAJORITY': # compare to voting threshold
            combined_mask = combined_mask >= majority_success
        
        combined_mask = combined_mask.astype(int) # convert boolean mask to integers. True/False = 1/0
        combined_mask = (combined_mask*255).astype(np.uint8) # scale to 0-255 in order to visualize
        combined_masks.append(combined_mask)
    
    if display_images:
        AF.plot_images(combined_masks, ['combined_masks ' + str(mission_numbers[i]) + '-' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)])
    
    temp_bounding_box_masks = []
    num_bbox = []
    
    for k in range(0, num_bbox_iterations):
        combined_bbox_masks = []

        for i in range(0, len(combined_masks)):
            label_ = label(combined_masks[i])
            props_ = regionprops(label_)
            bounding_mask = np.zeros((combined_masks[i].shape)).astype(np.uint8)

            if k == num_bbox_iterations-1:
                num_bbox.append(len(props_))

            factor = 0
            for prop in props_:
                if k == num_bbox_iterations-1:
                    cv2.rectangle(bounding_mask, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), 255, 2)
                else:
                    cv2.rectangle(bounding_mask, (prop.bbox[1]-factor, prop.bbox[0]-factor), (prop.bbox[3]+2*factor, prop.bbox[2]+2*factor), 255, -1)

            combined_bbox_masks.append(bounding_mask)

        temp_bounding_box_masks.append(copy.deepcopy(combined_bbox_masks))
        temp = copy.deepcopy(combined_bbox_masks)
    
    if display_images:
        for k in range(0, num_bbox_iterations):
            AF.plot_images(temp_bounding_box_masks[k], ['temp_bounding_box_masks ' + str(mission_numbers[i]) + '-' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)])
    
    # ----------------------------------------
    
    num_bboxs = [num_bbox[i] + num_bbox[i+1] for i in range(0, len(num_bbox)-1)]
    num_bboxs.insert(0, num_bbox[0])
    num_bboxs.append(num_bbox[-1])
    
    # ----------------------------------------
        
    colored_bbox_masks = []
    for i in range(0, len(combined_bbox_masks)):
        result = np.zeros((combined_bbox_masks[i].shape[0], combined_bbox_masks[i].shape[1], 3))
        result[combined_bbox_masks[i].astype(bool), :] = bbox_colors[0]
        colored_bbox_masks.append(result.astype(np.uint8))
        
    # display bbox 1 and 2 on image 1, display bbox 2 and 3 on image 2
    # display first set in red and second set in blue
    # 1-2, 1-2-3, 2-3-4, 4-5
    # 1    2    3    4
    superimposed_images = []
    for i in range(0, len(output_images)):
        result = copy.deepcopy(output_images[i])
        if i < len(output_images) - 1:
            result[combined_bbox_masks[i].astype(bool), :] = bbox_colors[0]
        if i > 0:
            result[combined_bbox_masks[i-1].astype(bool), :] = bbox_colors[1]
        superimposed_images.append(result.astype(np.uint8))
    
    return combined_bbox_masks, num_bboxs, colored_bbox_masks, superimposed_images

In [ ]:
def compute_results(mission_numbers, heatmaps, percent_heatmap_buckets, roof_masks, percent_roof_pixels, display_images=False, save_images=False):
    
    # ----------------------------------------
    
    # combine heatmap and roof mask into single image
    heatmap_roof_combined_images = []
    for i in range(0, len(mission_numbers)):
        heatmap_roof_combined_images.append(cv2.bitwise_or(heatmaps[i], roof_masks[i]).astype(np.uint8))
    
    if display_images:
        AF.plot_images(heatmap_roof_combined_images, ['heatmap_roof_combined_images ' + str(mission_number) for mission_number in mission_numbers])
    
    # ----------------------------------------
    
    # table of metadata or plots
    metadata = []
    for i in range(0, len(mission_numbers)):
        data = dict()
#         data['num_anomalies'] =  # TODO - from superimposed_bbox_masks
        data['percent_roof_anomalies'] = percent_roof_pixels[i]
        
#         heatmap_data = []
#         num_non_black_pixels = np.sum(heatmaps[i] != [0, 0, 0])
#         for c in heatmap_colors:
#             heatmap_data.append(np.sum(heatmaps[i] == c) / num_non_black_pixels * 100)
        data['heatmap_buckets'] = percent_heatmap_buckets[i]

        metadata.append(data)
    
    # ----------------------------------------
    
    return heatmap_roof_combined_images, metadata

In [ ]:
def display_results(mission_numbers, colored_bbox_images, superimposed_bbox_images, heatmap_roof_combined_images, metadata, color_buckets, num_bboxs, save_images=False):

    # display bounding boxes
    AF.plot_images(colored_bbox_images, ['Anomaly Bounding Boxes: Missions ' + str(mission_numbers[i]) + '-' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)], save_images=save_images, file_name='colored_bbox_images.png')
    AF.plot_images(superimposed_bbox_images, ['Superimposed Anomaly Bounding Boxes: Mission ' + str(mission_number) for mission_number in mission_numbers], save_images=save_images, file_name='superimposed_bbox_images.png')
    
    # display heatmap of grass regions scores with roof anomaly mask
    AF.plot_images(heatmap_roof_combined_images, ['Grass Score Heatmap: Mission ' + str(mission_number) for mission_number in mission_numbers], save_images=save_images, file_name='heatmap_roof_combined_images.png')
    
    # histogram of grass region scores
    barchart_data = [metadata[i]['heatmap_buckets'] for i in range(0 , len(mission_numbers))]
    AF.plot_images(barchart_data, title_list=['Histogram of Grass Scores: Mission ' + str(mission_number) for mission_number in mission_numbers], is_barchart=True, barchart_colors=color_buckets, grid='on')
    
    # scatter plot of percent anomaly pixels within roof image
    percent_roof_anomalies = [metadata[i]['percent_roof_anomalies'] for i in range(0, len(mission_numbers))]
    fig = plt.figure()
    plt.scatter(mission_numbers, percent_roof_anomalies, marker='o')
    plt.title('Anomaly Roof Percentage for each Mission')
    plt.xlabel('Mission Number')
    plt.ylabel('Anomaly Roof Percentage')
    plt.ylim([0, np.round(np.max(percent_roof_anomalies)*1.5)])
    plt.show()
    
    fig = plt.figure()
    plt.scatter(mission_numbers, num_bboxs, marker='o')
    plt.title('Number of Anomalies Detected for each Mission')
    plt.xlabel('Mission Number')
    plt.ylabel('Number of Anomalies Detected')
    plt.ylim([0, np.round(np.max(num_bboxs)*1.5)])
    plt.show()

In [ ]:
def run_visualize_results(mission_numbers, anomaly_masks, histogram_mission_images_bgr, num_bbox_iterations, bbox_colors, heatmaps, percent_heatmap_buckets, roof_masks, percent_roof_pixels, color_buckets, voting_scheme, majority_success, display_images=False, save_images=False):
    print('Visualize Results')
    print('Step 1 - merging anomaly masks')
    combined_bbox_masks, num_bboxs, colored_bbox_images, superimposed_bbox_images = create_bounding_boxes(mission_numbers, anomaly_masks, histogram_mission_images_bgr, num_bbox_iterations, bbox_colors, voting_scheme, majority_success, display_images)
    print('Step 2 - computing results')
    heatmap_roof_combinations, metadata = compute_results(mission_numbers, heatmaps, percent_heatmap_buckets, roof_masks, percent_roof_pixels, display_images)
    print('Step 3 - displaying results')
    display_results(mission_numbers, colored_bbox_images, superimposed_bbox_images, heatmap_roof_combinations, metadata, color_buckets, num_bboxs, save_images)